In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip freeze > requirements.txt

In [ ]:
!pip install pydub ffmpeg datasets transformers librosa noisereduce pymorphy2 SpeechRecognition accelerate -U

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 80.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 40.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 87.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cache

In [ ]:
#блок 1
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
from pathlib import Path
import librosa
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC

#агрузка данных из Excel без заголовков столбцов
df = pd.read_excel('/content/drive/MyDrive/rzd/norm.csv.xlsx', header=None)
transcriptions = df.iloc[:, :-1].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).tolist()
labels = df.iloc[:, -1].astype(int).tolist()

#в формат Dataset
data = Dataset.from_dict({'text': transcriptions, 'label': labels})

# Токенизация данных
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=512)
tokenized_data = data.map(tokenize_function, batched=True)

#Разделение данных на обучающую и тестовую выборки
train_dataset = tokenized_data.shuffle().select(range(int(0.8 * len(tokenized_data))))
test_dataset = tokenized_data.shuffle().select(range(int(0.8 * len(tokenized_data)), len(tokenized_data)))

#шаг 2: Обучение модели
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=2)
training_args = TrainingArguments(
    output_dir='/results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='/logs',
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

Map:   0%|          | 0/28 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# шанс

import speech_recognition as sr
from pathlib import Path
from transformers import BertTokenizer, BertForSequenceClassification
import torch
import re

#Инициализация распознавателя
recognizer = sr.Recognizer()

def transcribe_google(audio_path):
    with sr.AudioFile(audio_path) as source:
        audio_data = recognizer.record(source)
        text = recognizer.recognize_google(audio_data, language='ru-RU')
    return text

# Загрузка предобученной модели BERT для классификации текста
tokenizer_bert = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model_bert = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=2)

#для классификации транскрипции
def classify_transcription(transcription, tokenizer, model):
    inputs = tokenizer(transcription, return_tensors="pt", padding=True, truncation=True, max_length=512)
    outputs = model(**inputs)
    predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
    return torch.argmax(predictions, dim=-1).item()

#анализа транскрипции
def analyze_transcription(transcription):
  violations = []

  CRITERIA = {
       1: [
          # {"pattern": r'\b[А-Я][а-я]*[ов|ев|ин|ский|цкий|ая|яя|ой|ий]+\b', "meaning": "Отсутствие фамилии"},
           {"pattern": r'\bпозывной\b', "meaning": "Отсутствие упоминания позывного"},
           {"pattern": r'\bсокращение\b', "meaning": "Использование нестандартных сокращений не допускается"},
           {"pattern": r'\bсветофор\b', "meaning": "Не переданы показания светофоров по маршруту следования"}
       ],
       2: [
           {"pattern": r'\bпередача команды\b', "meaning": "Передача команды не лаконично"},
           {"pattern": r'\bздравствуйте|добрый день|пожалуйста|хорошо|до свидания|спасибо\b', "meaning": "Использование лишних слов"},
           {"pattern": r'\bверно\b', "meaning": "Отсутствие подтверждения правильности восприятия команды"}
       ],
       3: [
           {"pattern": r'\bрасстоянии до сцепления\b', "meaning": "Не передано сообщение о расстоянии до сцепления с вагонами"},
           {"pattern": r'\bизъятие тормозных башмаков\b', "meaning": "Передача разрешения на изъятие тормозных башмаков без доклада от машиниста"}
       ]
   }

  # Проверка на отсутствие фамилии
  if not re.search(r'\b[А-Я][а-я]*[ов|ев|ин|ский|цкий|ая|яя|ой|ий]+\b', transcription):
    violations.append((1, "Отсутствие фамилии"))

  # Критерий для использования лишних слов
#  extra_words_pattern = r'\b(здравствуйте|добрый день|пожалуйста|хорошо|до свидания|спасибо)\b'
 # if re.search(extra_words_pattern, transcription, re.IGNORECASE):
  #  violations.append((2, "Использование лишних слов"))

  for degree, criteria_list in CRITERIA.items():
       for criterion in criteria_list:
           if re.search(criterion['pattern'], transcription, re.IGNORECASE):
               violations.append((degree, criterion['meaning']))
  return violations

# Путь к папке с аудиофайлами
audio_folder_path = "/content/drive/My Drive/rzd/wav"
audio_files = list(Path(audio_folder_path).glob('*.wav'))

# Транскрибирование и классификация аудиофайлов
for audio_file in audio_files:
    transcription = transcribe_google(str(audio_file))
    classification = classify_transcription(transcription, tokenizer_bert, model_bert)
    violations = analyze_transcription(transcription)
    print(f"Файл: {audio_file.name}")
    print(f"Транскрипция: {transcription}")
    print(f"Классификация: {'Нарушение' if classification == 0 else 'Соответствие'}")
    if violations:
        for degree, meaning in violations:
            print(f"Нарушение степени {degree}: {meaning}")
    else:
        print("Транскрипция соответствует стандартам разговора.")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Файл: 66к_832 КВ - 02.05.2024 05_57_50.wav
Транскрипция: 2675 или даже Сибирская кравца Здравствуйте поездной диспетчер поездов
Классификация: Соответствие
Нарушение степени 2: Использование лишних слов
Файл: AUD-20240418-WA0001.wav
Транскрипция: на второй путь на выходе два закрыт на стоянку 2008 на 2 числа закрыт
Классификация: Соответствие
Нарушение степени 1: Отсутствие фамилии
Файл: AUD-20240418-WA0003.wav
Транскрипция: Фёдоров Здравствуйте Ракова прибывает на четвёртой свободный путь 107 Новатор
Классификация: Соответствие
Нарушение степени 2: Использование лишних слов
Файл: AUD-20240418-WA0004.wav
Транскрипция: 2822 негабарит 2200 Локомотив 4162200 8724 года
Классификация: Соответствие
Транскрипция соответствует стандартам разговора.
Файл: 57к_879 КВ - 02.05.2024 07_07_13.wav
Транскрипция: 4 392/3
Классификация: Нарушение
Нарушение степени 1: Отсутствие фамилии
Файл: 63к_824 КВ - 02.05.2024 04_21_45.wav
Транскрипция: 376 с одного 9766
Классификация: Соответствие
Нарушение степен